# Tune Tutorial

In [1]:
from helper import load_mnist_data

ModuleNotFoundError: No module named 'helper'

We want to start off by creating a model:

In [ ]:
def train_mnist(args):
    x_train, x_test, y_train, y_test = load_mnist_data()
    epochs = 12
    batch_size = 128
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(args.kernel1, args.kernel1),
                     activation='relu', input_shape=(28, 28, 1)))
    model.add(Conv2D(64, (args.kernel2, args.kernel2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(args.poolsize, args.poolsize)))
    model.add(Dropout(args.dropout1))
    model.add(Flatten())
    model.add(Dense(args.hidden, activation='relu'))
    model.add(Dropout(args.dropout2))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.SGD(
                      lr=args.lr, momentum=args.momentum),
                  metrics=['accuracy'])
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))

*Then*, we want to train this model (try out default hyperparameters)

Now, let's try running a simple search to find the best hyperparameters

In [ ]:
import argparse
parser = argparse.ArgumentParser(description='Keras MNIST Example')
parser.add_argument('--steps', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--kernel1', type=int, default=3,
                    help='Size of first kernel (default: 3)')
parser.add_argument('--kernel2', type=int, default=3,
                    help='Size of second kernel (default: 3)')
parser.add_argument('--poolsize', type=int, default=2,
                    help='Size of Pooling (default: 2)')
parser.add_argument('--dropout1', type=float, default=0.25,
                    help='Size of first kernel (default: 0.25)')
parser.add_argument('--hidden', type=int, default=128,
                    help='Size of Hidden Layer (default: 128)')
parser.add_argument('--dropout2', type=float, default=0.5,
                    help='Size of first kernel (default: 0.5)')

In [ ]:
train_mnist(args)

Now, let's use this machine with CPUs and multiplex our training to find the best parameters using a single machine.

In [ ]:
import ray
from ray import tune
from helper import TuneCallback

In [ ]:
def train_mnist(args, cfg, reporter):
    vars(args).update(cfg)
    x_train, x_test, y_train, y_test = load_data(limit_threads=4)
    
    epochs = 12
    batch_size = 128
    num_classes = 10
    
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(args.kernel1, args.kernel1),
                     activation='relu', input_shape=(28, 28, 1)))
    model.add(Conv2D(64, (args.kernel2, args.kernel2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(args.poolsize, args.poolsize)))
    model.add(Dropout(args.dropout1))
    model.add(Flatten())
    model.add(Dense(args.hidden, activation='relu'))
    model.add(Dropout(args.dropout2))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.SGD(
                      lr=args.lr, momentum=args.momentum),
                  metrics=['accuracy'])
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test),
              callbacks=[TuneCallback(reporter)])

In [ ]:
ray.init()
configuration = {"exp": {
        "stop": {
            "mean_accuracy": 0.99,
            "timesteps_total": 300
        },
        "run": lambda cfg, rprtr: train_mnist(args, cfg, rprtr),
#         "repeat": 100,
        "config": {
            "lr": lambda spec: np.random.uniform(0.001, 0.1),
            "momentum": lambda spec: np.random.uniform(0.1, 0.9),
            "hidden": lambda spec: np.random.randint(32, 512),
            "dropout1": lambda spec: np.random.uniform(0.2, 0.8),
        }
    }
}
tune.run_experiments(configuration)